## Modification: Using network analysis

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('dataset.csv', index_col=0)
id_list = df['user_id'].to_list()

In [2]:
df.head()

,user_id,tweets_count,followers_count,friends_count,has_name,has_image,has_address,has_bio,profile_has_url,present_in_list,friends_to_followers_ratio,label
0,3610511,3057,5470,2385,1,1,1,1,1,1,0.436015,1
1,5656162,3128,506,381,1,1,1,1,1,1,0.752964,1
2,5682702,3158,264,87,1,1,1,1,1,1,0.329545,1
3,6067292,3098,640,622,1,1,0,1,1,1,0.971875,1
4,6015122,2012,62,64,1,1,1,1,1,0,1.032258,1


In [3]:
import collections
import pandas as pd 

# a = [1,1,1,1,2,2,2,2,3,3,4,5,5]

# getting the follower count

df3 = pd.read_csv('followers.csv')
lst = df3['source_id'].to_list()
follower_count = collections.Counter(lst)
followers_count_sorted = sorted(follower_count.items())

# getting friends count
lst2 = df3['target_id'].to_list()
friend_count = collections.Counter(lst2)
friends_count_sorted = sorted(friend_count.items())

user_id_list = list(set(follower_count.keys()) | set(friend_count.keys()) | set(id_list))


In [4]:
df2 = pd.DataFrame({'user_id':sorted(user_id_list)})
df2['total_followers'] = [0] * len(user_id_list)
df2['total_friends'] = [0] * len(user_id_list)
df2.head(15)

,user_id,total_followers,total_friends
0,12,0,0
1,13,0,0
2,14,0,0
3,20,0,0
4,34,0,0
5,87,0,0
6,107,0,0
7,224,0,0
8,246,0,0
9,259,0,0


In [48]:
df_followers = pd.read_csv('followers.csv')

In [50]:
df_friends = pd.read_csv('friends.csv')

### Creating columns to be used for class C attributes

In [59]:
for ind in df_followers.index:
    account = df_followers['source_id'][ind]
    follower = df_followers['target_id'][ind]

    account_follower_count = follower_count[account]
    follower_follower_count = follower_count[follower]

    # df2[df2['user_id'] == account]['total_followers'] += follower_follower_count
    # df2[df2['user_id'] == follower]['total_followers'] += account_follower_count

    account_ind = df2.user_id[df2.user_id == account].index
    follower_ind = df2.user_id[df2.user_id == follower].index

    df2['total_followers'][account_ind] += follower_follower_count
    df2['total_followers'][follower_ind] += account_follower_count

In [60]:
for ind in df_followers.index:
    account = df_followers['source_id'][ind]
    follower = df_followers['target_id'][ind]

    account_friend_count = friend_count[account]
    follower_friend_count = friend_count[follower]

    # df2[df2['user_id'] == account]['total_followers'] += follower_follower_count
    # df2[df2['user_id'] == follower]['total_followers'] += account_follower_count

    account_ind = df2.user_id[df2.user_id == account].index
    follower_ind = df2.user_id[df2.user_id == follower].index

    df2['total_friends'][account_ind] += follower_friend_count
    df2['total_friends'][follower_ind] += account_friend_count

In [62]:
df2.head(30)
df2.to_csv('dataset-total.csv')

In [5]:
df_all = pd.read_csv('dataset-total.csv', index_col=0)
df_all.head(15)

,user_id,total_followers,total_friends
0,12,261,75164
1,13,109,46900
2,14,3,963
3,20,102,47151
4,34,1,5282
5,87,4,894
6,107,15,8191
7,224,6,2549
8,246,1,1060
9,259,1,1060


### Creating columns for class C attributes

In [6]:
df_all['mean_followers'] = [0] * len(user_id_list)
df_all['mean_friends'] = [0] * len(user_id_list)

In [7]:
df_all.head()

,user_id,total_followers,total_friends,mean_followers,mean_friends
0,12,261,75164,0,0
1,13,109,46900,0,0
2,14,3,963,0,0
3,20,102,47151,0,0
4,34,1,5282,0,0


In [9]:
for ind in df_all.index:

    account = df_all['user_id'][ind]
    if ind % 50000 == 0:
        print(ind)

    total_neighbours = 0    
    total_neighbours = friend_count[account] + follower_count[account]

    if total_neighbours == 0:
        df_all['mean_followers'][ind] = 0
        df_all['mean_friends'][ind] = 0 
    else:
        df_all['mean_followers'][ind] = df_all['total_followers'][ind] / total_neighbours
        df_all['mean_friends'][ind] = df_all['total_friends'][ind] / total_neighbours

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000


In [10]:
df_all.head(20)

,user_id,total_followers,total_friends,mean_followers,mean_friends
0,12,261,75164,6,1927
1,13,109,46900,9,3908
2,14,3,963,1,321
3,20,102,47151,7,3627
4,34,1,5282,1,5282
5,87,4,894,4,894
6,107,15,8191,5,2730
7,224,6,2549,6,2549
8,246,1,1060,1,1060
9,259,1,1060,1,1060


### Append these columns to the dataset

In [14]:
id_list[0]

3610511

In [58]:
# mean_followers_list = df_all['mean_followers'].to_list()
# mean_friends_list = df_all['mean_friends'].to_list()

# df['mean_followers'] = mean_followers_list
# df['mean_friends'] = mean_friends_list

# df.to_csv('dataset-2.csv')

mean_followers_list = [0] * len(id_list)
mean_friends_list = [0] * len(id_list)

k = 0

for u in id_list:
    if k % 100 == 0:
        print(u)

    account_ind = df_all.user_id[df_all.user_id == u].index
    mean_follower = df_all['mean_followers'][account_ind]
    mean_friend = df_all['mean_friends'][account_ind]
    mean_followers_list[k] = mean_follower.item()
    mean_friends_list[k] = mean_friend.item()
    k += 1


3610511
43964510
80000814
125088832
190569663
255656786
327397233
386779723
416351352
479832822
762202957
800130746
524820621
622239105
301919327
7773732
82062642
211462267
428434740
19861856
42570158
87752806
128595293
173215914
233688028
291362552
357101224
469178446
134435538
616220111
616227562
616297320
616312179
616636835
616914106
617262093
617557720
617828146
619266193
621554804
1207879530
1374959215
1127034602
1127827542
1129012411
1130518314
1174436491
1174680708
1174854703
1175061374
1175806116
1176043460
1176263858
1176392124


In [60]:
df.head()

,user_id,tweets_count,has_name,has_image,has_address,has_bio,profile_has_url,present_in_list,friends_to_followers_ratio
0,3610511,3057,1,1,1,1,1,1,0.436015
1,5656162,3128,1,1,1,1,1,1,0.752964
2,5682702,3158,1,1,1,1,1,1,0.329545
3,6067292,3098,1,1,0,1,1,1,0.971875
4,6015122,2012,1,1,1,1,1,0,1.032258


In [62]:
df['mean_followers_count'] = mean_followers_list

In [63]:
df.head(10)

,user_id,tweets_count,has_name,has_image,has_address,has_bio,profile_has_url,present_in_list,friends_to_followers_ratio,mean_followers_count
0,3610511,3057,1,1,1,1,1,1,0.436015,9
1,5656162,3128,1,1,1,1,1,1,0.752964,6
2,5682702,3158,1,1,1,1,1,1,0.329545,11
3,6067292,3098,1,1,0,1,1,1,0.971875,14
4,6015122,2012,1,1,1,1,1,0,1.032258,43
5,6140012,3203,1,1,1,1,0,1,1.297101,49
6,6134312,1183,1,1,1,1,0,1,1.312500,22
7,6684602,3102,1,1,0,1,0,1,1.666667,21
8,7046912,3113,1,1,1,1,1,1,0.040996,4
9,7470952,3186,1,1,1,1,0,1,0.936842,30


In [64]:
df['mean_friends_count'] = mean_friends_list
df['label'] = labels

In [65]:
df.head()

,user_id,tweets_count,has_name,has_image,has_address,has_bio,profile_has_url,present_in_list,friends_to_followers_ratio,mean_followers_count,mean_friends_count,label
0,3610511,3057,1,1,1,1,1,1,0.436015,9,151,1
1,5656162,3128,1,1,1,1,1,1,0.752964,6,6,1
2,5682702,3158,1,1,1,1,1,1,0.329545,11,9,1
3,6067292,3098,1,1,0,1,1,1,0.971875,14,82,1
4,6015122,2012,1,1,1,1,1,0,1.032258,43,12,1


In [66]:
df.tail()

,user_id,tweets_count,has_name,has_image,has_address,has_bio,profile_has_url,present_in_list,friends_to_followers_ratio,mean_followers_count,mean_friends_count,label
1164,1128385638,19,1,1,1,1,0,0,26.500000,17,0,0
1165,1128865231,20,1,1,1,1,0,0,23.555556,11,0,0
1166,1130046366,31,1,1,1,1,0,0,31.500000,9,0,0
1167,1176159360,15,1,1,1,1,0,0,20.636364,1,1,0
1168,1176392124,22,1,1,1,1,0,0,22.200000,7,0,0


In [67]:
df = df.drop(['friends_to_followers_ratio'], axis = 1)

In [68]:
df.head()

,user_id,tweets_count,has_name,has_image,has_address,has_bio,profile_has_url,present_in_list,mean_followers_count,mean_friends_count,label
0,3610511,3057,1,1,1,1,1,1,9,151,1
1,5656162,3128,1,1,1,1,1,1,6,6,1
2,5682702,3158,1,1,1,1,1,1,11,9,1
3,6067292,3098,1,1,0,1,1,1,14,82,1
4,6015122,2012,1,1,1,1,1,0,43,12,1


In [69]:
df.to_csv('dataset-graph.csv')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('dataset-graph.csv', index_col=0)

X = df.iloc[:,1:11]
y = df.iloc[:, -1]

X_real = X[:1950] 
y_real = y[:1950]

X_fake = X[1950:]
y_fake = y[1950:]

X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(X_real, y_real, test_size=0.2)
X_train_fake, X_test_fake, y_train_fake, y_test_fake = train_test_split(X_fake, y_fake, test_size=0.2)

frames = [X_train_real, X_train_fake]
X_train = pd.concat(frames)

frames = [y_train_real, y_train_fake]
y_train = pd.concat(frames)

frames = [X_test_real, X_test_fake]
X_test = pd.concat(frames)

frames = [y_test_real, y_test_fake]
y_test = pd.concat(frames)

### 1. Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
import numpy as np

clf = RandomForestClassifier(n_estimators = 100)   
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test) 

from sklearn import metrics   
print("Acc: ", metrics.accuracy_score(y_test, y_pred))

Acc:  1.0


### 2. Decision Tree

In [76]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

from sklearn import metrics   
print("Acc: ", metrics.accuracy_score(y_test, y_pred))

confusion_matrix(y_test, y_pred)

Acc:  1.0


array([[671,   0],
       [  0, 390]])

### Adaboost

In [77]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

clf = AdaBoostClassifier(n_estimators=50, learning_rate=1)

# Train Adaboost Classifer
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

from sklearn import metrics   
print("Acc: ", metrics.accuracy_score(y_test, y_pred))

Acc:  1.0


### SVM

In [78]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

from sklearn import metrics   
print("Acc: ", metrics.accuracy_score(y_test, y_pred))

Acc:  1.0
